In [ ]:
import itertools

In [ ]:
directory = '../datasets/'
name_file = 'accidents.dat'
path_file = directory + name_file
num_partition = 10

Qui va specificato il delimitatore per il file su cui si sta lavorando.

In [ ]:
data_file_delimiter = ' '

In [ ]:
file = sc.textFile(path_file,num_partition).map(lambda line: line.split())#.map(lambda item: map(int,item))

In [ ]:
%%time
file.collect()

In [ ]:
file.count()

In [ ]:
global global_supp_thresholds
global_supp_threshold = 0.4

In [ ]:
def get_frequent_singletons(baskets,local_threshold):
    """Funzione che prende calcola gli itemset di dimensione 1 che sono frequenti
    Input:
        - baskets, contiene gli itemset;
        - local_threshold, la threshold per filtrare gli item che hanno un supporto maggiore di essa.
    Output:
        - singletons, lista che contiene gli item frequenti per i basket passati in input.
    """
    candidates_dictionary = dict()
    singletons = list()

    for basket in baskets:
        for item in basket:
            if item not in candidates_dictionary.keys():
                candidates_dictionary[item] = 1
            else:
                candidates_dictionary[item] += 1
    
    for key,value in candidates_dictionary.items():
        if value >= local_threshold:
            singletons.append(key)
    
    return sorted(singletons)

In [ ]:
def getFrequents(baskets, prev_freq, dim,local_threshold):
    """Funzione la quale calcola gli itemset frequenti di dimensione maggiore di 1.
    Input:
        - basket, insieme di itemset su cui stiamo lavorando;
        - prev_freq, gli itemset frequenti di dimensione dim-1;
        - dim, la dimensione attuale;
        - local_threshold, minimo supporto che gli itemset devono superare o raggiungere per essere
            considerati frequenti
    Output:
        - freq_itemsets, lista degli itemset frequenti
    """
    candidates = list()
    dict_table = dict()
    freq_itemsets = list()
    if dim==2:
        table_C2 = itertools.combinations(prev_freq,dim)
        for element in table_C2:
            candidates.append(element)
    else:
        for item in itertools.combinations(prev_freq,2):
            if(len(set(item[0]).intersection(set(item[1]))) == dim-2):
                candidate_itemsets = tuple(set(item[0]).union(set(item[1])))
                if candidate_itemsets in candidates:
                    continue
                else:
                    pair = itertools.combinations(candidate_itemsets,dim-1)
                    
                    if set(pair).issubset(prev_freq):
                        candidates.append(candidate_itemsets)
                    
    for candidate in candidates:
        for basket in baskets:
            if set(candidate).issubset(set(basket)):
                dict_table.setdefault(candidate,0)
                dict_table[candidate] +=1

    for key,value in dict_table.items():
        if value >= local_threshold:
            freq_itemsets.append(key)

    return sorted(freq_itemsets)

In [ ]:
def apriori_algorithm(chunk):
    """Funzione che applica l'algoritmo APriori.
    Input:
        - chunk: partizione del file di input che contiene i basket
    Output:
        - freq_itemsets, lista degli itemset candidati dopo l'applicazione dell'algoritmo A-Priori
    """
    baskets = list()
    freq_itemsets = list()
    current_freq = list()
    for c in chunk:
        baskets.append(c)
    dimension = 1
    local_supp_threshold = len(baskets) * global_supp_threshold
    freq_singletons = get_frequent_singletons(baskets,local_supp_threshold)

    for single in freq_singletons:
        freq_itemsets.append(single)
    
    dimension +=1

    current_freq = freq_singletons
    
    while True:
        #print(f'curr freq: {current_freq}')
        prev_frequents = current_freq
        #print(f'prev_frequents: {prev_frequents}')
        current_freq = getFrequents(baskets,prev_frequents,dimension,local_supp_threshold)
        #print(f'curr freq dopo: {current_freq}')
        for item in current_freq:
            freq_itemsets.append(item)
        
        if len(current_freq) == 0:
            break
        dimension +=1

    return freq_itemsets


### Map Reduce
Viene applicato l'algoritmo **A Priori** per calcolare gli **itemset frequenti** dai quali poi otteniamo gli itemset candidati.

In [ ]:
candidatesItemsets = file.mapPartitions(apriori_algorithm).map(lambda item: (item,1))

In [ ]:
%%time
candidatesItemsets.count()

In [ ]:
%%time
candidatesItemsets_list = candidatesItemsets.collect()

In [ ]:
candidatesItemsets_list

In [ ]:
def countOccurences(chunk,candidates):
    '''Funzione che conta le occorrenze degli itemset all'interno del chunk 
        Input:
            - chunk, porzione del file di input;
            - candidates, lista che contine gli itemset.
        Output:
            - lista che contiene coppie chiave-valore dove la chiave è l'itemset e il valore è il suo conteggio
    '''
    itemsetOccurences = list()
    dictionary = dict()
    chunk_list = list(chunk)
    for basket in chunk_list:
        for candidate in candidates:
            if(set(candidate[0]).issubset(set(basket))):
                dictionary.setdefault(candidate[0],0)
                dictionary[candidate[0]] +=1
    
    for key,value in dictionary.items():
        itemsetOccurences.append((key,value)) 
    return itemsetOccurences

In [ ]:
output_SecMapReduce = file.mapPartitions(lambda chunk: countOccurences(chunk,candidatesItemsets_list)).reduceByKey(lambda a,b: a+b)

In [ ]:
output_SecMapReduce.collect()

In [ ]:
%%time
num_itemsetCandidates = output_SecMapReduce.count()
num_itemsetCandidates

In [ ]:
num_tot_baskets = file.count()
num_tot_baskets

Qui ho deciso di calcolare il supporto minimo andando a moltiplicare il numero di basket con la variabile globale.

In [ ]:
def normalize_values(item,min_value,max_value):
    '''Funzione che normalizza i valori:
        Input:
            - item, l'item da normalizzare che contiene chiave e valore;
            - min_value, il valore minimo negli itemset candidati;
            - max_value, il valore minimo negli itemset candidati.
        Output:
            - ritorna una tupla che contiene chiave e valore normalizzato
    '''
    norm_value = (item[1]-min_value)/(max_value-min_value)
    return (item[0],norm_value)

In [ ]:
values = output_SecMapReduce.map(lambda item: item[1])
min_value = values.min()
max_value = values.max()
normalized_secReduce = output_SecMapReduce.map(lambda item: normalize_values(item,min_value,max_value))
normalized_secReduce.collect()

In [ ]:
minSupportNormalized = global_supp_threshold
minSupportNormalized

In [ ]:
outputFinale_normalizzato = normalized_secReduce.filter(lambda item: item[1] >= minSupportNormalized)\
    .map(lambda item:item[0]).sortBy(lambda item: len(item))
   

In [ ]:
frequentItemset_normalizzato = outputFinale_normalizzato.collect()

In [ ]:
frequentItemset_normalizzato

Qua invece andiamo a calcolare i frequent itemset senza la normalizzazione e prendendo come supporto minimo il valore medio di tutti i valori nei candidati.

In [ ]:
def getminSuppwith_avg(candidates,num_candidates):
    '''Funzione che ritorna il supporto minimo andando
        a calcolare il valore medio.
        Input:
            - candidates, itemset candidati;
            - num_candidates;
        Outpur:
            supporto minimo'''
    sums = 0
    for c in candidates.collect():
        sums+= c[1]

    return int(sums/num_candidates)

In [ ]:
minSupport = getminSuppwith_avg(output_SecMapReduce,num_itemsetCandidates)
minSupport

In [ ]:
outputFinale = output_SecMapReduce.filter(lambda item: item[1] >= minSupport)\
    .map(lambda item:item[0]).sortBy(lambda item: len(item))

In [ ]:
%%time
frequentItemsets = outputFinale.collect()

In [ ]:
name = name_file.split('.')[0]

In [ ]:
#salavataggio dei frequent itemset senza la normalizzazione

f = open('./output/output_frequentItemsets_' + name + str(num_partition) +'.txt', 'w')
for item in frequentItemsets:
        if type(item) is tuple:
            length=len(item)
            f.write("(")
            for i in range(0,length-1):
                f.write(str(item[i])+",")
            f.write(str(item[length-1])+")\n")
        else:
            f.write(item + "\n")
f.close()

In [ ]:
#salavataggio dei frequent itemset con la normalizzazione

f = open('./output/output_frequentItemsetsNormalized_' + name + str(num_partition) +'.txt', 'w')
for item in frequentItemset_normalizzato:
        if type(item) is tuple:
            length=len(item)
            f.write("(")
            for i in range(0,length-1):
                f.write(str(item[i])+",")
            f.write(str(item[length-1])+")\n")
        else:
            f.write(item + "\n")
f.close()